## **Objetivo del Notebook**

Este proyecto se centra en el desarrollo de un Modelo de Machine Learning para la Predicción de Ictus, realizado por el departamento de Análisis de Datos del Hospital F5.

El modelo de predicción de ictus se basa en la utilización de 11 variables de información extraídas de un conjunto de datos que contiene 4,982 registros de casos positivos y negativos de ictus. Estas variables se utilizarán como características para entrenar y evaluar diversos algoritmos de clasificación de Machine Learning.

## **Contenido del Notebook**

0. Importación de librerías y dataset

1. Comprensión básica de los datos

2. Análisis exploratorio de datos (EDA)

3. Ingeniería de características

4. Preprocesamiento de datos

5. Entrenamiento del modelo

6. Comprobación del rendimiento del modelo

7. Ajuste de hiperparámetros del modelo

8. Análisis de las características con más peso en el modelo

9. Conclusiones

## **Importamos las librerías necesarias y el dataset**

In [ ]:
# Librerías de análisis
import numpy as np
import pandas as pd
import math
from scipy import stats

# Librerías de visualización


# Librerías de ML

In [1]:
import pandas as pd
path = "/work/stroke_dataset.csv"
df = pd.read_csv(path)

df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


## **1. Comprensión básica de los datos**

1.1. Descripción de los datos

1.2. Valores únicos de cada columna

1.3. Comprobación de las dimensiones de los datos

1.4. Visualización del conjunto de datos

1.5. Comprobación de tipos de datos

🔹 Categóricos

🔹 Numéricos

🔹 Tipos de datos mixtos

🔹 Errores o erratas
1.6. Comprobación de datos duplicados

1.7. Comprobación del número total y el porcentaje de valores omitidos

1.8. Comprobación de la cardinalidad de las características categóricas

### **1.1. Descripción de los datos**

### **1.2. Valores únicos de cada columna**

### **1.3. Comprobación de las dimensiones de los datos**

### **1.4. Visualización del conjunto de datos**

### **1.5. Comprobación de tipos de datos**

##### **Definición de variables categóricas y numéricas**

### **1.6. Comprobar los datos duplicados**

### **1.7. Verificar valores Nulos**

### **1.8. Comprobar la Cardinalidad**

**Cardinalidad de columnas numéricas**

**Cardinalidad de columnas categóricas**

## **2. Análisis exploratorio de datos (EDA)**


### **2.1. Analizamos la variable objetivo 'stroke'**

## **3. Ingeniería de características**


## **4. Preprocesamiento de datos**

4.1. Eliminación de columnas que no vamos a utilizar

4.2. Imputación de valores nulos

4.3. Codificación de características

4.4. Balanceo de características

4.5. Selección de las características y la variable objetivo

4.6. División en train y test


4.5. Escalado de características

### **4.1. Eliminación de columnas que no vamos a utilizar**

### **4.2. Imputación de valores nulos**

### **4.3. Codificación de características**

### **4.4. Balanceo de características**

### **4.5. Selección de las características y la variable objetivo**

### **4.6. División en train y test**

### **4.7. Escalado de características**

## **5. Entrenamiento del modelo**

## **6. Comprobación del rendimiento del modelo con validación cruzada**

## **7. Ajuste de hiperparámetros del modelo**

## **8. Análisis de las características con más peso en el modelo**

## **9. Conclusiones**

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=311bc360-0ff3-42ad-8fe8-990672a12326' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>